<a href="https://colab.research.google.com/github/leoquiroa/DeepLearning/blob/master/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import plot_precision_recall_curve

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import GridSearchCV

%tensorflow_version 1.x

In [4]:
import tensorflow
print(tensorflow.__version__)

1.15.0


# Data Preparing

Data preprocessing

In [5]:
url = "https://raw.githubusercontent.com/leoquiroa/DeepLearning/master/Data/Churn_Modelling.csv"
dataset = pd.read_csv(
    url, 
    names = ["RowNumber","CustomerId","Surname","CreditScore","Geography","Gender","Age","Tenure","Balance","NumOfProducts","HasCrCard","IsActiveMember","EstimatedSalary","Exited"], 
    skiprows=1)
print(dataset.shape)
dataset.head()

(10000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
X = dataset.iloc[:, 3:13].values # CreditScore, Geography, Gender, Age, Tenure, Balance, NumOfProducts,	HasCrCard, IsActiveMember, EstimatedSalary
y = dataset.iloc[:, 13].values #Exited
print(X.shape)
print(y.shape)

(10000, 10)
(10000,)


In [7]:
X[1:10,:]

array([[608, 'Spain', 'Female', 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [502, 'France', 'Female', 42, 8, 159660.8, 3, 1, 0, 113931.57],
       [699, 'France', 'Female', 39, 1, 0.0, 2, 0, 0, 93826.63],
       [850, 'Spain', 'Female', 43, 2, 125510.82, 1, 1, 1, 79084.1],
       [645, 'Spain', 'Male', 44, 8, 113755.78, 2, 1, 0, 149756.71],
       [822, 'France', 'Male', 50, 7, 0.0, 2, 1, 1, 10062.8],
       [376, 'Germany', 'Female', 29, 4, 115046.74, 4, 1, 0, 119346.88],
       [501, 'France', 'Male', 44, 4, 142051.07, 2, 0, 1, 74940.5],
       [684, 'France', 'Male', 27, 2, 134603.88, 1, 1, 1, 71725.73]],
      dtype=object)

Two categorical variables: Geography and Gender. Apply Label-Encoder.

In [8]:
dataset['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [9]:
dataset['Gender'].value_counts()

Male      5457
Female    4543
Name: Gender, dtype: int64

In [0]:
# Encoding categorical data (Geography and Gender)
labelencoder_1 = LabelEncoder()
X[:, 1] = labelencoder_1.fit_transform(X[:, 1])
labelencoder_2 = LabelEncoder()
X[:, 2] = labelencoder_2.fit_transform(X[:, 2])

In [11]:
X[1:10,:]

array([[608, 2, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [502, 0, 0, 42, 8, 159660.8, 3, 1, 0, 113931.57],
       [699, 0, 0, 39, 1, 0.0, 2, 0, 0, 93826.63],
       [850, 2, 0, 43, 2, 125510.82, 1, 1, 1, 79084.1],
       [645, 2, 1, 44, 8, 113755.78, 2, 1, 0, 149756.71],
       [822, 0, 1, 50, 7, 0.0, 2, 1, 1, 10062.8],
       [376, 1, 0, 29, 4, 115046.74, 4, 1, 0, 119346.88],
       [501, 0, 1, 44, 4, 142051.07, 2, 0, 1, 74940.5],
       [684, 0, 1, 27, 2, 134603.88, 1, 1, 1, 71725.73]], dtype=object)

Create dummy variables for the Geography (since it has > 2 categories) with One-Hot-Encoder

In [0]:
A = make_column_transformer(
    (OneHotEncoder(categories='auto'), [1]), 
    remainder="passthrough")
X=A.fit_transform(X)

In [13]:
X[1:10,:]

array([[0.0, 0.0, 1.0, 608, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [1.0, 0.0, 0.0, 502, 0, 42, 8, 159660.8, 3, 1, 0, 113931.57],
       [1.0, 0.0, 0.0, 699, 0, 39, 1, 0.0, 2, 0, 0, 93826.63],
       [0.0, 0.0, 1.0, 850, 0, 43, 2, 125510.82, 1, 1, 1, 79084.1],
       [0.0, 0.0, 1.0, 645, 1, 44, 8, 113755.78, 2, 1, 0, 149756.71],
       [1.0, 0.0, 0.0, 822, 1, 50, 7, 0.0, 2, 1, 1, 10062.8],
       [0.0, 1.0, 0.0, 376, 0, 29, 4, 115046.74, 4, 1, 0, 119346.88],
       [1.0, 0.0, 0.0, 501, 1, 44, 4, 142051.07, 2, 0, 1, 74940.5],
       [1.0, 0.0, 0.0, 684, 1, 27, 2, 134603.88, 1, 1, 1, 71725.73]],
      dtype=object)

Geography
* France (0)
* Germany (1)
* Spain (2)
* Remove France

Gender
* Male (1)
* Female (0)

Avoid the Dummy variable trap by removing the first column. Now, it has only two columns for the country.

In [0]:
X = X[:, 1:]

In [15]:
X[1:10,:]

array([[0.0, 1.0, 608, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [0.0, 0.0, 502, 0, 42, 8, 159660.8, 3, 1, 0, 113931.57],
       [0.0, 0.0, 699, 0, 39, 1, 0.0, 2, 0, 0, 93826.63],
       [0.0, 1.0, 850, 0, 43, 2, 125510.82, 1, 1, 1, 79084.1],
       [0.0, 1.0, 645, 1, 44, 8, 113755.78, 2, 1, 0, 149756.71],
       [0.0, 0.0, 822, 1, 50, 7, 0.0, 2, 1, 1, 10062.8],
       [1.0, 0.0, 376, 0, 29, 4, 115046.74, 4, 1, 0, 119346.88],
       [0.0, 0.0, 501, 1, 44, 4, 142051.07, 2, 0, 1, 74940.5],
       [0.0, 0.0, 684, 1, 27, 2, 134603.88, 1, 1, 1, 71725.73]],
      dtype=object)

Splitting the dataset into the Training set and Test set

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
print(X_train.shape)
print(X_test.shape)

(8000, 11)
(2000, 11)


Feature Scaling. We don't one independent variable dominating another one.

In [0]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [18]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(10000, 11)
(8000, 11)
(2000, 11)


In [19]:
print(min(X[:,2]))
print(max(X[:,2]))
print(min(X_train[:,2]))
print(max(X_train[:,2]))
print(min(X_test[:,2]))
print(max(X_test[:,2]))

350
850
-3.0983464242408694
2.0561144379132976
-2.943712598376244
2.0561144379132976


# Model

Initialize the DL model
1. Defining the sequence of layers (*)
2. Defining a graph

Activation Functions
* Relu for hidden layers
* Sigmoid as output (probabilities)
* Softmax for more than two categories

**Back-Propagation:**
Update the weights according to how much they are responsible for the error

**The Learning Rate:**
Decides how much we update the weights

Update the weights after observations
* one (Reinforcement Learning)
* batch (Batch Learning)

**Number of nodes**
* Art: average of input+output
* Not Art: Grid search for parameter tunning 

**Optimizer**, the algorithm to find the optimal set of weights in the NN. Stochastic Gradient Descent. (i.e. Adam)

**Loss**
* categorical_crossentropy > 2 categories has outcome
* binary_crossentropy = binary outcome

**Batch size**
The number of observations after which you want to update the weight

In [0]:
classifier = Sequential()
# input layer and first hidden layer
classifier.add(
    Dense(
        kernel_initializer='uniform'
        ,input_dim=11
        ,activation='relu'
        ,units=6
        )
    )
# second hidden layer
classifier.add(
    Dense(
        kernel_initializer='uniform'
        ,units=6
        ,activation='relu'
        )
    )
# output layer
classifier.add(
    Dense(
        kernel_initializer='uniform'
        ,units=1
        ,activation='sigmoid'
        )
    )
#compile
classifier.compile(
    optimizer='adam'
    ,loss='binary_crossentropy'
    ,metrics = ['accuracy']
    )






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
classifier.fit(x=X_train,y=y_train,batch_size=10,epochs=100)




Epoch 1/100





8000/8000 [==============================] - 1s 187us/step - loss: 0.4839 - acc: 0.7954
Epoch 2/100
8000/8000 [==============================] - 1s 115us/step - loss: 0.4268 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 117us/step - loss: 0.4220 - acc: 0.7990
Epoch 4/100
8000/8000 [==============================] - 1s 120us/step - loss: 0.4184 - acc: 0.8210
Epoch 5/100
8000/8000 [==============================] - 1s 116us/step - loss: 0.4162 - acc: 0.8262
Epoch 6/100
8000/8000 [==============================] - 1s 115us/step - loss: 0.4144 - acc: 0.8287
Epoch 7/100
8000/8000 [==============================] - 1s 123us/step - loss: 0.4131 - acc: 0.8315
Epoch 8/100
8000/8000 [==============================] - 1s 116us/step - loss: 0.4122 - acc: 0.8324
Epoch 9/100
8000/8000 [==============================] - 1s 117us/step - loss: 0.4104 - acc: 0.8320
Epoch 10/100
8000/8000 [==============================] - 1s 116us/step - loss: 0.4101 - acc

In [0]:
y_pred = classifier.predict(X_test)
print(y_pred.shape)
y_pred[1:10,:]

(2000, 1)


array([[0.3122647 ],
       [0.16596654],
       [0.04954922],
       [0.16718063],
       [0.87469333],
       [0.12094805],
       [0.06944406],
       [0.16374594],
       [0.87506545]], dtype=float32)

* The percentage indicates the probability that certain customer is most likely to leave the bank.
* Segment the 10% of their customers with the highest probabilities and then analyze them in more depth with data-mining techniques.

In [0]:
threshold = 0.5
y_pred = (y_pred>threshold)

In [0]:
cm = confusion_matrix(y_test,y_pred)
cm

array([[1548,   47],
       [ 253,  152]])

In [0]:
print('accuracy_score: ',accuracy_score(y_test, y_pred))
print('precision_score: ',precision_score(y_test, y_pred))
print('recall_score: ',recall_score(y_test, y_pred))
print('f1_score: ',f1_score(y_test, y_pred))

accuracy_score:  0.85
precision_score:  0.7638190954773869
recall_score:  0.37530864197530867
f1_score:  0.5033112582781457


In [0]:
average_precision = average_precision_score(y_test, y_pred)
print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

Average precision-recall score: 0.41


# Prediction

Predict if the customer with the following informations will leave the bank: 
* Geography: France
* Credit Score: 600
* Gender: Male
* Age: 40 years old
* Tenure: 3 years
* Balance: 60,000
* Number of Products: 2
* Does this customer have a credit card ? Yes
* Is this customer an Active Member: Yes
* Estimated Salary: $50000

In [0]:
new_prediction = classifier.predict(
    sc.transform(
        np.array(
            [[
              0.0    # Geography: Germany
              ,0     # Geography: Spain
              ,600   # Credit Score
              ,1     # Gender: Male
              ,40    # Age 
              ,3     # Tenure
              ,60000 # Balance
              ,2     # number of products
              ,1     # has credit card
              ,1     # active member
              ,50000 # estimated salary
            ]]
          )
        )
    )

In [0]:
dataset.iloc[1, 3:13]

CreditScore            608
Geography            Spain
Gender              Female
Age                     41
Tenure                   1
Balance            83807.9
NumOfProducts            1
HasCrCard                0
IsActiveMember           1
EstimatedSalary     112543
Name: 1, dtype: object

In [0]:
new_prediction

array([[0.13707006]], dtype=float32)

In [0]:
# Trye (leaves the bank)
# False (doesn't leave the bank, the person stays)
new_prediction = (new_prediction > 0.5) 
new_prediction

array([[False]])

# Evaluating the ANN with K Cross validation

In [0]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [0]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

In [25]:
print(accuracies)
print(mean)
print(variance)

[0.83749999 0.85249999 0.87749999 0.83125    0.84749999 0.83125
 0.86124999 0.82375    0.81       0.85124999]
0.8423749943822623
0.018611571248312406


# Grid Search

In [0]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [0]:
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'epochs': [10, 25],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)

In [29]:
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/10





7200/7200 [==============================] - 1s 88us/step - loss: 0.5647 - acc: 0.7965
Epoch 2/10
7200/7200 [==============================] - 0s 50us/step - loss: 0.4359 - acc: 0.7971
Epoch 3/10
7200/7200 [==============================] - 0s 47us/step - loss: 0.4311 - acc: 0.7971
Epoch 4/10
7200/7200 [==============================] - 0s 49us/step - loss: 0.4285 - acc: 0.7971
Epoch 5/10
7200/7200 [==============================] - 0s 52us/step - loss: 0.4264 - acc: 0.7971
Epoch 6/10
7200/7200 [==============================] - 0s 49us/step - loss: 0.4241 - acc: 0.7971
Epoch 7/10
7200/7200 [==============================] - 0s 54us/step - loss: 0.4220 - acc: 0.7971
Epoch 8/10
7200/7200 [==============================] - 0s 50us/step - loss: 0.4205 - acc: 0.8092
Epoch 9/10
7200/7200 [==============================] - 0s 51us/step - loss: 0.4189 - acc: 0.8194
Epoch 10/10

In [30]:
grid_search

GridSearchCV(cv=10, error_score=nan,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7fcd2f649da0>,
             iid='deprecated', n_jobs=None,
             param_grid={'batch_size': [25, 32], 'epochs': [10, 25],
                         'optimizer': ['adam', 'rmsprop']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [31]:
grid_search.best_params_

{'batch_size': 32, 'epochs': 25, 'optimizer': 'rmsprop'}

In [32]:
grid_search.best_score_

0.8385000000000001